In [7]:
import sqlite3
import os

# Connexion à la base de données (le fichier .db sera créé si il n'existe pas)
conn = sqlite3.connect('infractions.db')

# Création d'un curseur pour exécuter des requêtes SQL
cursor = conn.cursor()

# Création des tables
cursor.execute('''
CREATE TABLE IF NOT EXISTS infractions (
    code_infr INT PRIMARY KEY,
    lib_infr VARCHAR(50)
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS départements (
    code_dep VARCHAR(50) PRIMARY KEY,
    nom_dep VARCHAR(50) NOT NULL
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS force_odre (
    force_odre VARCHAR(50) PRIMARY KEY
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS annee (
    annee VARCHAR(50) PRIMARY KEY
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS périmètres (
    lib_perimetre VARCHAR(50) PRIMARY KEY,
    force_odre VARCHAR(50) NOT NULL,
    FOREIGN KEY(force_odre) REFERENCES force_odre(force_odre)
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS brigade (
    id_brigade VARCHAR(50) PRIMARY KEY,
    lib_brigade VARCHAR(50) NOT NULL,
    lib_perimetre VARCHAR(50),
    code_dep VARCHAR(50) NOT NULL,
    force_odre VARCHAR(50) NOT NULL,
    FOREIGN KEY(lib_perimetre) REFERENCES périmètres(lib_perimetre),
    FOREIGN KEY(code_dep) REFERENCES départements(code_dep),
    FOREIGN KEY(force_odre) REFERENCES force_odre(force_odre)
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS fait (
    code_infr INT,
    id_brigade VARCHAR(50),
    annee VARCHAR(50),
    nb_infraction INT,
    PRIMARY KEY(code_infr, id_brigade, annee),
    FOREIGN KEY(code_infr) REFERENCES infractions(code_infr),
    FOREIGN KEY(id_brigade) REFERENCES brigade(id_brigade),
    FOREIGN KEY(annee) REFERENCES annee(annee)
)
''')

# Enregistrement des modifications
conn.commit()

# Fermeture de la connexion
conn.close()

print("Base de données 'infractions.db' créée avec succès!")

Base de données 'infractions.db' créée avec succès!


On va créer le dataframe qui servira à alimenter la base

In [2]:
def create_df(file_path):
    # Charger le fichier Excel
    xls = pd.ExcelFile(file_path)
    
    # Prendre uniquement les feuilles commençant par "Services"
    sheets = [sheet for sheet in xls.sheet_names if sheet.startswith("Services")]
    
    # Liste pour stocker les données
    data = []
    
    # Parcourir les feuilles
    for sheet in sheets:
        # Identifier l'année et la direction
        libelle_annee = sheet[-4:]
        if "PN" in sheet:
            nom_organisation = "Police Nationale"
            nom_organisation_court = "PN"
        else:
            nom_organisation = "Gendarmerie Nationale"
            nom_organisation_court = "GN"
        
        # Charger les données
        df = pd.read_excel(xls, sheet_name=sheet, header=None)
        
        # Définir les indices de départ selon PN ou GN
        row_start = 3 if "GN" in sheet else 4
        col_start = 2  # Les données commencent en colonne C
        
        # Extraire les valeurs spécifiques pour les ID
        code_departement = df.iloc[0, col_start:]  # Ligne 1, colonnes à partir de C
        id_perimetre = df.iloc[1, col_start:] if "PN" in sheet else pd.Series([""] * (df.shape[1] - col_start))
        id_cgd_csp = df.iloc[2, col_start:] if "PN" in sheet else df.iloc[1, col_start:]
        
        # Extraire les ID infractions et les nombres de faits
        id_infraction = df.iloc[row_start - 1:, 0].reset_index(drop=True)  # Colonne A
        lib_infraction = df.iloc[row_start - 1:, 1].reset_index(drop=True)  # Colonne B
        nombre_faits = df.iloc[row_start - 1:, col_start:].reset_index(drop=True)  # Table des faits
        
        # Transformer les données en format exploitable
        for col in range(nombre_faits.shape[1]):
            for row in range(nombre_faits.shape[0]):
                data.append({
                    "lib_annee": libelle_annee,
                    "nom_organisation": nom_organisation,
                    "code_departement": code_departement.iloc[col],
                    "lib_perimetre": id_perimetre.iloc[col],
                    "lib_cgd_csp" : id_cgd_csp.iloc[col],
                    "id_organisation" : nom_organisation_court + id_cgd_csp.iloc[col] + id_perimetre.iloc[col],
                    "id_infraction": id_infraction.iloc[row],
                    "lib_infraction": lib_infraction.iloc[row],
                    "nombre_faits": nombre_faits.iloc[row, col]
                })

    # Ajouter un identifiant avec l'index du DataFrame
    data_df = pd.DataFrame(data)
    data_df.reset_index(inplace=True)

    return data_df 

In [3]:
import pandas as pd

# On crée le DF sur la base du fichier Excel avec le format crée au dessus
file_path = "crimes-et-delits-enregistres-par-les-services-de-gendarmerie-et-de-police-depuis-2012.xlsx"
data_df = create_df(file_path)

In [4]:
data_df.head()

,index,lib_annee,nom_organisation,code_departement,lib_perimetre,lib_cgd_csp,id_organisation,id_infraction,lib_infraction,nombre_faits
0,0,2012,Police Nationale,01,DCPAF,DIDPAF PREVESSIN 01,PNDIDPAF PREVESSIN 01DCPAF,1,Règlements de compte entre malfaireurs,0.0
1,1,2012,Police Nationale,01,DCPAF,DIDPAF PREVESSIN 01,PNDIDPAF PREVESSIN 01DCPAF,2,Homicides pour voler et à l'occasion de vols,0.0
2,2,2012,Police Nationale,01,DCPAF,DIDPAF PREVESSIN 01,PNDIDPAF PREVESSIN 01DCPAF,3,Homicides pour d'autres motifs,0.0
3,3,2012,Police Nationale,01,DCPAF,DIDPAF PREVESSIN 01,PNDIDPAF PREVESSIN 01DCPAF,4,Tentatives d'homicides pour voler et à l'occas...,0.0
4,4,2012,Police Nationale,01,DCPAF,DIDPAF PREVESSIN 01,PNDIDPAF PREVESSIN 01DCPAF,5,Tentatives homicides pour d'autres motifs,0.0


In [6]:
import sqlite3
import pandas as pd

# Connexion à la base de données SQLite
conn = sqlite3.connect('infractions.db')
cursor = conn.cursor()

# Vérifier si "nom_dep" est disponible, sinon remplir par défaut
if 'nom_dep' not in data_df.columns:
    data_df['nom_dep'] = 'Inconnu'  # Valeur par défaut pour éviter les erreurs SQL

# Extraire les départements uniques avec code et nom
departements = data_df[['code_departement', 'nom_dep']].drop_duplicates()

# Renommer les colonnes pour correspondre à la table SQL
departements_renamed = departements.rename(columns={
    'code_departement': 'code_dep'
})

# Insérer dans la table départements
cursor.executemany("INSERT OR IGNORE INTO départements (code_dep, nom_dep) VALUES (?, ?)", departements_renamed.values)

# Insérer les autres données (restant inchangées)
annees = data_df[['lib_annee']].drop_duplicates()
cursor.executemany("INSERT OR IGNORE INTO annee (annee) VALUES (?)", annees.values)

forces_odre = data_df[['nom_organisation']].drop_duplicates()
cursor.executemany("INSERT OR IGNORE INTO force_odre (force_odre) VALUES (?)", forces_odre.values)

infractions = data_df[['id_infraction', 'lib_infraction']].drop_duplicates()
cursor.executemany("INSERT OR IGNORE INTO infractions (code_infr, lib_infr) VALUES (?, ?)", infractions.values)

perimetres = data_df[['lib_perimetre', 'nom_organisation']].drop_duplicates()
cursor.executemany("INSERT OR IGNORE INTO périmètres (lib_perimetre, force_odre) VALUES (?, ?)", perimetres.values)

brigades = data_df[['id_organisation', 'lib_cgd_csp', 'lib_perimetre', 'code_departement', 'nom_organisation']].drop_duplicates()
brigades_renamed = brigades.rename(columns={
    'id_organisation': 'id_brigade',
    'lib_cgd_csp': 'lib_brigade',
    'code_departement': 'code_dep',
    'nom_organisation': 'force_odre'
})
cursor.executemany("INSERT OR IGNORE INTO brigade (id_brigade, lib_brigade, lib_perimetre, code_dep, force_odre) VALUES (?, ?, ?, ?, ?)", brigades_renamed.values)

faits = data_df[['id_infraction', 'id_organisation', 'lib_annee', 'nombre_faits']]
faits_renamed = faits.rename(columns={
    'id_infraction': 'code_infr',
    'id_organisation': 'id_brigade',
    'lib_annee': 'annee',
    'nombre_faits': 'nb_infraction'
})
cursor.executemany("INSERT OR IGNORE INTO fait (code_infr, id_brigade, annee, nb_infraction) VALUES (?, ?, ?, ?)", faits_renamed.values)

# Validation des changements
conn.commit()
conn.close()

print("Données insérées avec succès dans 'infractions.db' !")


Données insérées avec succès dans 'infractions.db' !
